# Import Libraries

In [ ]:
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests 
from bs4 import BeautifulSoup
import re

In [ ]:
import pandas as pd
import numpy as np

# Instantiate Model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# Test Model

In [ ]:
token = tokenizer.encode('Really nice food, shame about the hair in the dish though.', return_tensors='pt')

In [ ]:
result = model(token)

In [ ]:
result.logits

In [ ]:
int(torch.argmax(result.logits))+1

# Scrape Reviews

In [ ]:
yelp_link = r'https://www.yelp.co.uk/biz/the-pantry-amsterdam'

In [ ]:
r = requests.get(yelp_link)
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [ ]:
reviews[0]

In [ ]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [ ]:
df

In [ ]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [ ]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [ ]:
sentiment_score(df['review'].iloc[0])

In [ ]:
df['review'].iloc[2]

# Scrape Profile Names

In [ ]:
r = requests.get(yelp_link)
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*user.*')
results = soup.find_all('a', {'href':regex})
profiles = [result.text for result in results]

In [ ]:
profile_names = []
for name in profiles:
    if ". " in name:
        continue
    if "." in name:
        profile_names.append(name)

In [ ]:
o = 1
for i in profile_names:
    print(str(o), ')', i )
    o = o + 1

In [ ]:
profile_names

In [ ]:
df['name'] = profile_names

# Scrape Review Date

In [ ]:
r = requests.get(yelp_link)
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*css-chan6m.*')
results = soup.find_all('span', {'class':regex})
review_date = [result.text for result in results]

In [ ]:
df['date'] = review_date[:10]

# Add ID & Save to CSV

In [ ]:
df['bar_id'] = 1

In [ ]:
scraped_reviews = df.reindex(columns=['bar_id', 'date', 'name', 'review', 'sentiment'])
scraped_reviews

In [ ]:
scraped_reviews.to_csv(r'C:\Users\jesse\Sentiment Analysis Project\reviews.csv', index=0)